In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8m-pose.pt")

from torchinfo import summary 
summary(model)

import torch
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

model = model.to(DEVICE)

import cv2 
capture = cv2.VideoCapture("/kaggle/input/yolo8-squart/squart.mp4")

capture.get(cv2.CAP_PROP_FRAME_COUNT)


from ultralytics.utils.plotting import Annotator
def draw_keypoints(result, frame):
    tmp_pose = {}
    global nkps
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        kps = kps.data.squeeze()
        annotator.kpts(kps)
       
        nkps = kps.cpu().numpy()
        # nkps[:,2] = 1
        # annotator.kpts(nkps)
        tmp_pose['왼쪽엉덩이']   = nkps[11]
        tmp_pose['오른쪽엉덩이'] = nkps[12]
        for idx, (x, y, score) in enumerate(nkps):
            if score > 0.5:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
    pose.append([tmp_pose])
    return frame



def predict(frame, iou=0.7, conf=0.25):
    results = model(source=frame,
            device='cpu',
            iou=iou ,
            conf=conf ,
            verbose=False,
            )
    return results[0]


def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame


capture = cv2.VideoCapture("/kaggle/input/yolo8-squart/squart.mp4")
pose = []
while True:
    ret, frame = capture.read()
    # print(capture.get(cv2.CAP_PROP_POS_FRAMES))
    state_frame = capture.get(cv2.CAP_PROP_POS_FRAMES)
    if state_frame % 30 == 0:
        print("저장")
        result = predict(frame)
    # frame = draw_boxes(result, frame)
        frame = draw_keypoints(result, frame)
    # frame = cv2.flip(frame, 1)


    # cv2.putText(frame, text, position, font, scale, color, thickness)
    if not ret:
        print("영상 오류")
        capture.release()
        cv2.destroyAllWindows()
        break
